# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

***

## Part III: Update Information and Mapping

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


***
## Import Preprocessed Datasets

In [4]:
# Creates a DataFrame from reading a csv file 
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv"))
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP,4000.0,ml
1,P-26184,BAKED|Beans,9.000,Kg,PREP,9000.0,g
2,P-54644,BAKED|Moussaka,16.000,PTN,NaN,16.0,PTN
3,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN,6176.0,g
4,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN,7360.0,g


In [5]:
# Creates a DataFrame from reading a csv file 
ghge_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "ghge_factors.csv"))
ghge_factors.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [6]:
# Creates a DataFrame from reading a csv file 
nitro_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "nitrogen_factors.csv"))
nitro_factors.head()

,Category ID,Food Category,g N lost/kg product
0,1,beef & buffalo meat,329.50
1,2,lamb/mutton & goat meat,231.15
2,3,pork (pig meat),132.80
3,4,"poultry (chicken, turkey)",116.80
4,5,butter,100.35


In [7]:
# Creates a DataFrame from reading a csv file 
water_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "water_factors.csv"))
water_factors.head()

,Category ID,Food Category,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,1,beef & buffalo meat,1677.200,61309.000
1,2,lamb/mutton & goat meat,461.200,258.900
2,3,pork (pig meat),1810.300,54242.700
3,4,"poultry (chicken, turkey)",370.300,333.500
4,5,butter,1010.176,50055.168


In [8]:
# Creates a DataFrame from reading a csv file 
land_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "land_factors.csv"))
land_factors.rename(columns={'km^2 land use/kg product': 'Land Use (m^2)'}, inplace=True)
land_factors['Land Use (m^2)'] *= 1000
land_factors.head()

,Category ID,Food Category,Land Use (m^2)
0,1,beef & buffalo meat,0.12645
1,2,lamb/mutton & goat meat,0.14320
2,3,pork (pig meat),0.02102
3,4,"poultry (chicken, turkey)",0.01151
4,5,butter,0.01395


In [9]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [10]:
new_items = pd.read_csv("data/mapping/new items added/New_Items_2024/New_Items+Added_2024-08-09.csv")
new_items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3310,54,BASIL LEAF*LG SKR,1.00,LG SKR,775.000,g,SPICES
1,I-72216,1,BEEF STEW MEAT TRNB,1.00,lb,1.000,lb,MEAT
2,I-72465,21,CEREAL GRANOLA*PLAIN,3.00,bag,1.000,Kg,FOOD - GROCERY
3,I-4009,4,CHICK THIGH B/LS/L DICED,1.00,Kg,1.000,Kg,POULTRY
4,I-13634,12,COD GREY 3oz LOIN IQF MSC,1.00,lb,1.000,lb,SEAFOOD
5,I-70559,53,COFFEE COLD BREW,2.00,JUG,1.000,JUG,BEVERAGE
6,I-65800,8,CREAM 18%*946ML,1.00,ea,946.000,ml,DAIRY
7,I-71938,41,FRIES 3/8IN*CRINKLE,6.00,bag,5.000,lb,PRODUCE
8,I-73346,41,FRIES SPIRAL*REDSTONE,6.00,bag,4.000,lb,PRODUCE
9,I-71889,41,FRIES WAFFLE*SPICY CRUNCH,6.00,bag,4.000,lb,PRODUCE


In [11]:
item_nonstd = pd.read_csv("data/cleaning/Items_Nonstd.csv")
item_nonstd

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [12]:
preps_nonstd = pd.read_csv("data/cleaning/Preps_NonstdUom.csv")
preps_nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom


In [13]:
conv_updatecov = pd.read_csv("data/cleaning/update/Conv_UpdateConv.csv")
conv_updatecov

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398000,g
1,I-28697,0.005181,1.0,ea,193.000000,g
2,I-47441,0.005181,1.0,ea,193.000000,g
3,I-1905,0.012500,1.0,CT,80.000000,g
4,I-47440,0.005051,1.0,ea,198.000000,g
...,...,...,...,...,...,...
616,I-30911,0.058824,3.0,each,51.000000,g
617,I-72009,0.022046,1.0,ea,45.360000,g
618,I-18791,0.013228,3.0,CT,226.795833,g
619,I-1151,0.013889,1.0,CT,72.000000,g


In [14]:
conv_updatecov.loc[conv_updatecov["ConversionId"] == "P-54581"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [15]:
Preps.loc[Preps["PrepId"] == "P-54581"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
576,P-54581,SLICE|Multigrain Bread,22.0,slice,NaN,22.0,slice


# Update Conversion_Added.csv

In [16]:
conversions_added = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv"))
conversions_added

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.000000,L
1,NaN,0.877193,1.0,1.14L,1.140000,L
2,NaN,0.666667,1.0,1.5L,1.500000,L
3,NaN,0.571429,1.0,1.75 L,1.750000,L
4,NaN,0.500000,1.0,2L,2.000000,L
...,...,...,...,...,...,...
680,I-30911,0.058824,3.0,each,51.000000,g
681,I-72009,0.022046,1.0,ea,45.360000,g
682,I-18791,0.013228,3.0,CT,226.795833,g
683,I-1151,0.013889,1.0,CT,72.000000,g


In [17]:
# Concatenate the DataFrames
combined_df = pd.concat([conv_updatecov, conversions_added], ignore_index=True)
combined_df

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398000,g
1,I-28697,0.005181,1.0,ea,193.000000,g
2,I-47441,0.005181,1.0,ea,193.000000,g
3,I-1905,0.012500,1.0,CT,80.000000,g
4,I-47440,0.005051,1.0,ea,198.000000,g
...,...,...,...,...,...,...
1301,I-30911,0.058824,3.0,each,51.000000,g
1302,I-72009,0.022046,1.0,ea,45.360000,g
1303,I-18791,0.013228,3.0,CT,226.795833,g
1304,I-1151,0.013889,1.0,CT,72.000000,g


In [18]:
# Add the conv_updatecov file to the masterfile, Conversions_Added
path = os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv")
combined_df.to_csv(path, index = False, header = True)

***
## Import Update Info

In [19]:
# Import list of prep that need convert uom to standard uom manually
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))
# Here we can see that UOM examples are: each, ea, slice
Manual_PrepU.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.0,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.0,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.0,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.0,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.0,g


In [20]:
# The row of the DatFrame that contains the PrepId == P-54581
Manual_PrepU.loc[Manual_PrepU["PrepId"] == "P-54581"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
212,P-54581,SLICE|Multigrain Bread,22.0,slice,NaN,38.0,g


In [21]:
# Select the file path for new items list with category id
New_Items_Added = pd.read_csv("data/mapping/new items added/New_Items_2024/New_Items_Added_2024-08-13.csv")
New_Items_Added.tail(15)

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
41,I-71745,3,SALAMI GENO*SLICES,6.0,pak,500.00,g,DELI & PREPARED MEAT
42,I-71851,58,SAUCE CARAMEL*,2.0,JUG,4.00,L,FOOD - GROCERY
43,I-73610,58,SAUCE MAKHANI SIMMERING,1.0,10LT,10.00,L,FOOD - GROCERY
44,I-3530,58,SAUCE OYSTER*,2.0,4L,4.00,L,FOOD - GROCERY
45,I-3531,58,SAUCE PIZZA*,6.0,LG CAN,2.84,L,FOOD - GROCERY
46,I-16496,58,SAUCE PLUM*ULTIMATE,2.0,4L,4.00,L,FOOD - GROCERY
47,I-73411,54,SEASONING SALT GRANDPA J,2.0,BG,5.00,Kg,SPICES
48,I-40942,26,SEED CHIA WHITE,5000.0,g,1.00,g,SPICES
49,I-72201,1,SOUP BEEF*BARLEY MUSHROOM,3.0,BG,4.00,L,FOOD - GROCERY
50,I-72196,22,SOUP GOODLY*CORN CHOWDER SPICY,3.0,BG,4.00,L,FOOD - GROCERY


In [22]:
# Import list of items that adjusted GHGe factor manually
Manual_Factor = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Manual_Adjust_Factors.csv"))
Manual_Factor['Land Use (m^2)'] = 0
Manual_Factor.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU),Land Use (m^2)
0,I-52090,59.0,BURGER BEEF & MUSHROOM HALAL,1.0,cs,48.00,CT,MEAT,25.00894,200.86,1038.84,37961.2,0
1,I-45558,59.0,Prep-Vegan Parmesan,1000.0,g,1.00,g,PRODUCTION FOOD,3.85686,0.00,0.00,0.0,0
2,I-3352,59.0,MAYONNAISE PAIL TFC 4L,2.0,each,4.00,L,FOOD - GROCERY,3.55000,0.00,0.00,0.0,0
3,I-3223,59.0,COCONUT MILK 17/19% MILK FAT,6.0,LG CAN,2.84,L,FOOD - GROCERY,3.50000,0.00,1.00,1.0,0
4,I-2898,59.0,MUSTARD DIJON WINE FLEUR,6.0,jar,1.00,Kg,FOOD - GROCERY,3.32600,0.00,0.00,0.0,0


### Update Correct Uom for Preps

In [23]:
# Update prep list with manually adjusted uom
for index, row in Manual_PrepU.iterrows():
    PrepId = Manual_PrepU.loc[index, 'PrepId']
    qty = Manual_PrepU.loc[index, 'StdQty']
    uom = Manual_PrepU.loc[index, 'StdUom']
    Preps.loc[Preps['PrepId'] == PrepId, 'StdQty'] = qty
    Preps.loc[Preps['PrepId'] == PrepId, 'StdUom'] = uom

In [24]:
Preps.drop_duplicates(subset=['PrepId'], inplace=True,)

In [25]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP,4000.0,ml
1,P-26184,BAKED|Beans,9.000,Kg,PREP,9000.0,g
2,P-54644,BAKED|Moussaka,16.000,PTN,NaN,6720.0,g
3,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN,6176.0,g
4,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN,7360.0,g


In [26]:
Preps.shape

(778, 7)

In [27]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_List_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

In [28]:
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [29]:
New_Items_Added.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-73836,1,BEEF GRIND 85/15 NATURAL,1.0,Kg,1.0,Kg,MEAT
1,I-73834,1,BEEF STIRFRY GRASS FED,1.0,pak,1.0,lb,MEAT
2,I-73835,1,BEEF STRIPLOIN BNL 63 ACRES,1.0,Kg,1.0,Kg,MEAT
3,I-72009,20,BEYOND MEAT*SAUSAGE PATTY1.6oz,1.0,cs,96.0,ea,MEAT
4,I-73349,44,BUTTER PLANT*BASED VEGAN,6.0,TUB,340.0,g,DAIRY


### Import List of New Items with Emission Factors Category ID Assigned

In [30]:
frames = [Items_Assigned, New_Items_Added]
Items_Assigned_Updated = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()
Items_Assigned_Updated.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [31]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,NaN
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,NaN
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,NaN
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,NaN


In [32]:
Items_Assigned_Updated.shape

(2967, 9)

In [33]:
# Double brackets used to specify the column as a dataframe and not a series
# converting the 'CategoryID' column in the Items_Assigned_Updated dataframe to numeric data type using the pd.to_numeric()
# use the apply column to apply the coversion to the entire column
Items_Assigned_Updated[['CategoryID']] = Items_Assigned_Updated[['CategoryID']].apply(pd.to_numeric)

In [34]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,NaN
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,NaN
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,NaN
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,NaN


In [35]:
path = os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv")
Items_Assigned_Updated.to_csv(path, index = False, header = True)

## Mapping Items to Footprint Factors

In [36]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,NaN
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,NaN
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,NaN
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,NaN


In [37]:
# DEFINITION OF MAPPING: assigning certain attributes to data points based on criteria or predefined rules.

# Map GHG footprint factors
# merges two data frames Items_Assigned_Updated and ghge_factors on the columns CategoryID and Category ID, respectively. The 
# resulting data frame is stored in mapping.

# how='left' specifies that a left join is done between Items_Assigned_Updated and ghge_factors.
# left_on='CategoryID' specifies that the join condition for Items_Assigned_Updated should be based on the 'CategoryID' column.
# right_on='Category ID' specifies that the join condition for ghge_factors should be based on the 'Category ID' column.

mapping = pd.merge(Items_Assigned_Updated, ghge_factors.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')
# if row in mapping has CategoryId == NaN then it assigns the value of the column Active Total Supply Chain Emissions (kg CO2 / kg food)
# to zero
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0
# drops the columns Category ID and Food Category
display(mapping)
mapping = mapping.drop(columns=['Category ID', 'Food Category_x'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_x,Category ID,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,NaN,1.0,beef & buffalo meat,41.3463
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,NaN,1.0,beef & buffalo meat,41.3463
...,...,...,...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,NaN,8.0,cream,6.9824
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,NaN,40.0,other vegetables,0.5029
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,NaN,54.0,stimulants & spices misc.,9.3703
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,NaN,24.0,"wheat/rye (bread, pasta, baked goods)",1.5225


,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463
...,...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,cream,6.9824
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,other vegetables,0.5029
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,stimulants & spices misc.,9.3703
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,"wheat/rye (bread, pasta, baked goods)",1.5225


In [38]:
# Map nitrogen footprint factors
mapping = pd.merge(mapping, nitro_factors.loc[:,['Category ID','Food Category','g N lost/kg product']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'g N lost/kg product'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50
...,...,...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,cream,6.9824,28.08
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,other vegetables,0.5029,7.90
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,stimulants & spices misc.,9.3703,6.75
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,"wheat/rye (bread, pasta, baked goods)",1.5225,14.80


In [39]:
# Map land footprint factors
mapping = pd.merge(mapping, land_factors.loc[:,['Category ID','Food Category','Land Use (m^2)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Land Use (m^2)'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
...,...,...,...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,cream,6.9824,28.08,0.00780
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,other vegetables,0.5029,7.90,0.00119
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,stimulants & spices misc.,9.3703,6.75,0.03100
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,"wheat/rye (bread, pasta, baked goods)",1.5225,14.80,0.00489


In [40]:
# Map water footprint factors
# mapping: DataFrame that is used to assign sustainability-related factors (greenhouse gas emissions, nitrogen loss, and 
# water footprint) to food items based on their category and other attributes. 
mapping = pd.merge(mapping, water_factors.loc[:,['Category ID','Food Category','Freshwater Withdrawals (L/FU)', 'Stress-Weighted Water Use (L/FU)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Freshwater Withdrawals (L/FU)'] = 0
        mapping.loc[index,'Stress-Weighted Water Use (L/FU)'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping.drop_duplicates(subset=["ItemId"], inplace=True)
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,cream,6.9824,28.08,0.00780,28.08,30502.368
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,other vegetables,0.5029,7.90,0.00119,81.30,2939.500
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,stimulants & spices misc.,9.3703,6.75,0.03100,24.90,220.300
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,"wheat/rye (bread, pasta, baked goods)",1.5225,14.80,0.00489,419.20,12821.700


In [41]:
mapping[mapping["ItemId"] == "I-4524"]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
2006,I-4524,38.0,BEETS GOLDBC,5.0,lb,1.0,lb,PRODUCE,root vegetables,0.3062,7.9,0.00032,9.9,37.9


In [42]:
mapping["CategoryID"].isnull().sum()

0

### Manully Adjust Footprint Factor for Specific Items

In [43]:
# For Manual_Factor: dataframe with items that adjusted GHGe factor manually.
# It takes the id for Manual_Factor for the index being iterated and if it is equal to item id of mapping dataframe then it sets
# the values of the columns to the manually adjusted values
# Note: the values for the columns in mapping DataFrame is adjusted and not Manual_Factor DataFrame
for index, row in Manual_Factor.iterrows():
    itemId = Manual_Factor.loc[index, 'ItemId']
    ghge = Manual_Factor.loc[index, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
    nitro = Manual_Factor.loc[index, 'g N lost/kg product']
    water = Manual_Factor.loc[index, 'Freshwater Withdrawals (L/FU)']
    land = Manual_Factor.loc[index, 'Land Use (m^2)']
    str_water = Manual_Factor.loc[index, 'Stress-Weighted Water Use (L/FU)']
    mapping.loc[mapping['ItemId'] == itemId, 'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = ghge
    mapping.loc[mapping['ItemId'] == itemId, 'g N lost/kg product'] = nitro
    mapping.loc[mapping['ItemId'] == itemId, 'Freshwater Withdrawals (L/FU)'] = water
    mapping.loc[mapping['ItemId'] == itemId, 'Stress-Weighted Water Use (L/FU)'] = str_water
    mapping.loc[mapping['ItemId'] == itemId, 'Land Use (m^2)'] = land

In [44]:
mapping.drop_duplicates(subset = ['ItemId'], inplace=True)
mapping.dtypes

ItemId                                                     object
CategoryID                                                float64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Food Category_y                                            object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
g N lost/kg product                                       float64
Land Use (m^2)                                            float64
Freshwater Withdrawals (L/FU)                             float64
Stress-Weighted Water Use (L/FU)                          float64
dtype: object

In [45]:
mapping.shape

(2967, 14)

In [46]:
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.00,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.00,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.20,61309.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,I-72412,8.0,SOUP GOODLY*CREAMY TOMATO,3.00,bag,4.00,L,FOOD - GROCERY,cream,6.9824,28.08,0.00780,28.08,30502.368
2963,I-56665,40.0,SPRING ROLL*VEG MINI 27G,2.00,bag,48.00,ea,FOOD - GROCERY,other vegetables,0.5029,7.90,0.00119,81.30,2939.500
2964,I-73413,54.0,STEAK RUB GRANDPA J,2.00,BG,2.27,Kg,SPICES,stimulants & spices misc.,9.3703,6.75,0.03100,24.90,220.300
2965,I-38431,24.0,STUFFING BREAD,7.00,Kg,1.00,Kg,BREAD,"wheat/rye (bread, pasta, baked goods)",1.5225,14.80,0.00489,419.20,12821.700


In [47]:
ingredients = pd.read_csv("data/preprocessed/Ingredients_List.csv")
ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-48933,1.00,Kg,1.00000,1.0000,P-10113
1,P-18746,1.00,Kg,1.00000,1.0000,P-10241
2,I-3388,1.00,L,1.00000,0.3058,P-10496
3,I-4658,2.27,Kg,2.20462,0.6942,P-10496
4,I-12176,4.00,g,0.00100,0.3636,P-12112
...,...,...,...,...,...,...
5122,P-26068,10.00,g,0.00100,1.0000,R-51258
5123,P-26205,5.00,g,0.00100,1.0000,R-51258
5124,P-32747,100.00,ml,0.00100,1.0000,R-51258
5125,P-50837,5.00,g,1.00000,1.0000,R-51258


In [48]:
map_list = mapping["ItemId"].unique()
absent_list = []

for item in ingredients["IngredientId"].unique():
    if item not in map_list and item.startswith("I-"):
        absent_list.append(item)

# absent_list contains the IngredientIds that are not present in mapping but are present in the ingredients DataFrame
print(absent_list)

[]


In [49]:
# Converts mapping DataFrame to the Mapping.csv file
path = os.path.join(os.getcwd(), "data", "mapping", "Mapping.csv")
mapping.to_csv(path, index = False, header = True)